# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 21 2023 8:30AM,257783,10,0086408824,ISDIN Corporation,Released
1,Feb 21 2023 8:30AM,257783,10,0086408836,ISDIN Corporation,Released
2,Feb 21 2023 8:30AM,257783,10,0086412812,ISDIN Corporation,Released
3,Feb 21 2023 8:30AM,257784,10,0086414578,ISDIN Corporation,Released
4,Feb 21 2023 8:30AM,257784,10,0086414586,ISDIN Corporation,Released
5,Feb 21 2023 8:30AM,257783,10,0086408795,ISDIN Corporation,Released
6,Feb 21 2023 8:30AM,257783,10,0086408799,ISDIN Corporation,Released
7,Feb 21 2023 8:30AM,257783,10,0086408800,ISDIN Corporation,Released
8,Feb 21 2023 8:30AM,257783,10,0086408811,ISDIN Corporation,Released
9,Feb 21 2023 8:30AM,257783,10,0086408812,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,257782,Released,58
30,257783,Released,10
31,257784,Released,2
32,257786,Released,1
33,257790,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257782,NaN,NaN,58.0
257783,NaN,NaN,10.0
257784,NaN,NaN,2.0
257786,NaN,NaN,1.0
257790,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257690,0.0,0.0,1.0
257695,0.0,0.0,1.0
257696,0.0,0.0,1.0
257697,0.0,0.0,1.0
257705,1.0,3.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257690,0,0,1
257695,0,0,1
257696,0,0,1
257697,0,0,1
257705,1,3,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257690,0,0,1
1,257695,0,0,1
2,257696,0,0,1
3,257697,0,0,1
4,257705,1,3,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257690,,,1
1,257695,,,1
2,257696,,,1
3,257697,,,1
4,257705,1,3,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation
3,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation
11,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation
70,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated"
71,Feb 21 2023 8:30AM,257786,10,"Citieffe, Inc."
72,Feb 20 2023 5:06PM,257790,16,"Virtue Rx, LLC"
73,Feb 20 2023 2:48PM,257775,10,ISDIN Corporation
74,Feb 20 2023 1:43PM,257768,10,ISDIN Corporation
76,Feb 20 2023 1:40PM,257765,10,"Snap Medical Industries, LLC"
81,Feb 20 2023 12:01PM,257752,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,,,10
1,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,,,2
2,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,,,58
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",,,1
4,Feb 21 2023 8:30AM,257786,10,"Citieffe, Inc.",,,1
5,Feb 20 2023 5:06PM,257790,16,"Virtue Rx, LLC",,,1
6,Feb 20 2023 2:48PM,257775,10,ISDIN Corporation,,,1
7,Feb 20 2023 1:43PM,257768,10,ISDIN Corporation,,,2
8,Feb 20 2023 1:40PM,257765,10,"Snap Medical Industries, LLC",,,5
9,Feb 20 2023 12:01PM,257752,10,"Snap Medical Industries, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10,,
1,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2,,
2,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58,,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1,,
4,Feb 21 2023 8:30AM,257786,10,"Citieffe, Inc.",1,,
5,Feb 20 2023 5:06PM,257790,16,"Virtue Rx, LLC",1,,
6,Feb 20 2023 2:48PM,257775,10,ISDIN Corporation,1,,
7,Feb 20 2023 1:43PM,257768,10,ISDIN Corporation,2,,
8,Feb 20 2023 1:40PM,257765,10,"Snap Medical Industries, LLC",5,,
9,Feb 20 2023 12:01PM,257752,10,"Snap Medical Industries, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10,,
1,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2,,
2,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58,,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1,,
4,Feb 21 2023 8:30AM,257786,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10.0,NaN,NaN
1,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2.0,NaN,NaN
2,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58.0,NaN,NaN
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Feb 21 2023 8:30AM,257786,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10.0,0.0,0.0
1,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2.0,0.0,0.0
2,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58.0,0.0,0.0
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Feb 21 2023 8:30AM,257786,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6443546,402.0,48.0,5.0
16,515532,1.0,1.0,0.0
22,773088,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6443546,402.0,48.0,5.0
1,16,515532,1.0,1.0,0.0
2,22,773088,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,402.0,48.0,5.0
1,16,1.0,1.0,0.0
2,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,402.0
1,16,Released,1.0
2,22,Released,3.0
3,10,Executing,48.0
4,16,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,22
Status,,,
Completed,5.0,0.0,0.0
Executing,48.0,1.0,0.0
Released,402.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,22
0,Completed,5.0,0.0,0.0
1,Executing,48.0,1.0,0.0
2,Released,402.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,22
0,Completed,5.0,0.0,0.0
1,Executing,48.0,1.0,0.0
2,Released,402.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()